In [1]:
import ray

In [2]:
ray.init()

2024-12-26 16:05:03,829	INFO worker.py:1636 -- Connecting to existing Ray cluster at address: 10.128.0.31:6379...
2024-12-26 16:05:04,234	INFO worker.py:1812 -- Connected to Ray cluster. View the dashboard at http://127.0.0.1:8265 


Python version:,3.12.8
Ray version:,2.40.0
Dashboard:,http://127.0.0.1:8265


(remote_exec_func pid=155400) FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
(remote_exec_func pid=155400) FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
(remote_exec_func pid=155400) FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
(remote_exec_func pid=155400) FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
(remote_exec_func pid=155400) FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
(remote_exec_func pid=155400) FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
(remote_exec_func pid=155400) FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
(remote_exec_func pi

In [3]:
pip install ucimlrepo


Note: you may need to restart the kernel to use updated packages.


In [11]:
from ucimlrepo import fetch_ucirepo 
import modin.pandas as pd

import modin.config as cfg
cfg.Engine.put('ray')
cfg.StorageFormat.put('pandas')

In [6]:
df = ray.data.read_bigquery(project_id = 'double-fusion-445405-r6',query = 'SELECT * FROM l1_tridorian.wholesale_customer')

2024-12-26 16:09:04,906	INFO bigquery_datasource.py:68 -- Created streams: 1
2024-12-26 16:09:04,907	INFO bigquery_datasource.py:70 -- The number of streams created by the BigQuery Storage Read API is less than the requested parallelism due to the size of the dataset.


In [8]:
dfm = df.to_modin()

2024-12-26 16:09:33,391	INFO bigquery_datasource.py:68 -- Created streams: 1
2024-12-26 16:09:33,394	INFO bigquery_datasource.py:70 -- The number of streams created by the BigQuery Storage Read API is less than the requested parallelism due to the size of the dataset.
2024-12-26 16:09:33,405	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-12-25_17-21-23_032831_9010/logs/ray-data
2024-12-26 16:09:33,407	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadBigQuery]


Running 0: 0.00 row [00:00, ? row/s]

2024-12-26 16:09:36,172	INFO bigquery_datasource.py:68 -- Created streams: 1
2024-12-26 16:09:36,173	INFO bigquery_datasource.py:70 -- The number of streams created by the BigQuery Storage Read API is less than the requested parallelism due to the size of the dataset.


- ReadBigQuery->SplitBlocks(200) 1: 0.00 row [00:00, ? row/s]

In [24]:
## make the target is [0, 1, 2] instead of [1, 2, 3]
dfm['Region'] = dfm['Region'] - 1

dfm.head()


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,1,0,30624,7209,4897,18711,763,2876
1,1,0,11686,2154,6824,3527,592,697
2,1,0,9670,2280,2112,520,402,347
0,1,0,25203,11487,9490,5065,284,6854
1,1,0,583,685,2216,469,954,18


In [18]:
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb
from ray import train, tune

#, test_x, train_y, test_y = 

In [23]:
train_y.value_counts()

the groupby keys will be sorted anyway, although the 'sort=False' was passed. See the following issue for more details: https://github.com/modin-project/modin/issues/3571.


Region
3    238
1     58
2     34
Name: count, dtype: int64

In [143]:
from xgboost_ray import RayDMatrix, RayParams, predict


def data_prep(df):
    df = df.drop_duplicates()
    train_x, test_x_1, train_y, test_y_1 = train_test_split(dfm.drop('Region', axis = 1), dfm['Region'], test_size=0.4)
    valid_x, test_x, valid_y, test_y = train_test_split(test_x_1, test_y_1, test_size=0.4)
    return (train_x,valid_x, test_x, train_y, valid_y, test_y)

train_x,valid_x, test_x, train_y, valid_y, test_y = data_prep(dfm)

def train_wholesale_customer(config):
    # Load dataset
    # Split into train and test set
    # Build input matrices for XGBoost
    train_set = xgb.DMatrix(train_x, label=train_y)
    valid_set = xgb.DMatrix(valid_x, label=valid_y)
    # Train the classifier
    results = {}
    xgb.train(
        config,
        train_set,
        evals=[(valid_set, "eval")],
        evals_result=results,
        verbose_eval=False,
    )
    # Return prediction accuracy
    accuracy = 1.0 - results["eval"]["merror"][-1]
    train.report({"mean_accuracy": accuracy, "done": True})

config = {
    "objective": "multi:softmax",
    "num_class":3,
    "eval_metric": ["mlogloss", "merror"],
    "max_depth": tune.randint(1, 9),
    "n_estimators" : tune.randint(100, 1000),
    "min_child_weight": tune.choice([1, 2, 3]),
    "subsample": tune.uniform(0.5, 1.0),
    "eta": tune.loguniform(1e-4, 1e-1),
}
tuner = tune.Tuner(
    train_wholesale_customer,
    tune_config=tune.TuneConfig(num_samples=10),
    param_space=config,
)
results = tuner.fit()

2024-12-26 17:39:18,127	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/zaenal/ray_results/train_wholesale_customer_2024-12-26_17-38-01' in 0.0182s.
2024-12-26 17:39:18,149	INFO tune.py:1041 -- Total run time: 77.10 seconds (77.03 seconds for the tuning loop).


In [103]:
best_modl = results.get_best_result(metric = 'mean_accuracy', mode = 'max')

In [144]:
train_set = xgb.DMatrix(train_x, label=train_y)
valid_set = xgb.DMatrix(valid_x, label=valid_y)
clf = xgb.train(
    best_modl.config,
    train_set,
    evals=[(valid_set, "eval")],
    verbose_eval=True,
)

[0]	eval-mlogloss:1.09846	eval-merror:0.36190
[1]	eval-mlogloss:1.09833	eval-merror:0.33333
[2]	eval-mlogloss:1.09820	eval-merror:0.33333
[3]	eval-mlogloss:1.09805	eval-merror:0.33333
[4]	eval-mlogloss:1.09791	eval-merror:0.33333
[5]	eval-mlogloss:1.09777	eval-merror:0.33333
[6]	eval-mlogloss:1.09765	eval-merror:0.33333
[7]	eval-mlogloss:1.09750	eval-merror:0.33333
[8]	eval-mlogloss:1.09736	eval-merror:0.33333
[9]	eval-mlogloss:1.09721	eval-merror:0.33333


In [ ]:
## Save the best model

In [145]:
clf.save_model("model_wholesale_customer.xgb")

In [ ]:
### Check on the testing a

In [174]:

data = xgb.DMatrix(test_x)

bst = xgb.Booster(model_file="model.xgb")
(test_y == bst.predict(data)).sum()/test_x.shape[0]

0.676056338028169